In [10]:
from multiprocessing import Process
from utils import getMODISLAI
import ee
def f(num):
    return num * num

if __name__ == '__main__':
    # for i in range(0,10,1):
    ee.Initialize()
    p = Process(target=getMODISLAI, args=(60,-120,61,-118,2010,))
    p.start()
    p.join()
    print(p)

<Process name='Process-18' pid=59258 parent=59033 stopped exitcode=0>


In [1]:
import rioxarray
year = 2000
nfis_tif = rioxarray.open_rasterio(f'/Users/gclyne/Downloads/CA_forest_VLCE2_{year}/CA_forest_VLCE2_{year}.tif',decode_coords='all')


In [4]:
import multiprocessing
from utils import getMODISLAI,elevation
import time


if __name__ == "__main__":
    start = time.time()
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    jobs = []

    p = multiprocessing.Process(target=getMODISLAI, args=(60,-120,61,-118,2010,0, return_dict))
    p1 = multiprocessing.Process(target=elevation, args=(60,-120,61,-118,1, return_dict))
    p2 = multiprocessing.Process(target=getNFISAverage, args=(60,-120,61,-118,2, return_dict))
    jobs.append(p)
    jobs.append(p1)
    jobs.append(p2)
    p.start()
    p1.start()
    p2.start()

    for proc in jobs:
        proc.join()
    print(return_dict.values())
    print(f'multi took {time.time()  - start} seconds')

[412.19900257006236, 7.162171245093047, 58.681575856809395]
multi took 21.560444831848145 seconds


In [1]:
from utils import getMODISLAI,elevation,getNFISAverage
import time

start = time.time()
getNFISAverage(60,-120,61,-118,2,{})
getMODISLAI(60,-120,61,-118,2010,0, {})
elevation(60,-120,61,-118,0,{})
print(f'single took {time.time() - start} seconds')

single took 17.71528387069702 seconds


In [1]:
import csv 
boreal_coordinates = []
ordered_latitudes = []
ordered_longitudes = []

with open('/Users/gclyne/thesis/boreal_latitudes_longitudes.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        boreal_coordinates.append((float(row[0]),float(row[1])))



In [2]:
x = iter(boreal_coordinates)
lat = 60
lon = -120
next_lat = 61
next_lon = -118
year = 2000
def getCoordinates(latlon):
    lat = latlon[0]
    lon = latlon[1]
    next_lat = lat + 1
    next_lon = lon + 1
    return lat,lon,next_lat,next_lon

In [4]:
import multiprocessing
import csv
import rioxarray
from generate_observable_rows import getRow
import xarray as xr
if __name__ == '__main__':
    file = open('test.csv','w')
    writer = csv.writer(file)
    p = multiprocessing.Pool(10)
    nfis_tif = rioxarray.open_rasterio(f'/Users/gclyne/Downloads/CA_forest_VLCE2_{year}/CA_forest_VLCE2_{year}.tif',decode_coords='all',lock=False)
    # era_temp = xr.open_dataset('/Users/gclyne/thesis/data/_2m_temperature.nc',lock=False)

    with p: 
        for i in range(20):
            lat,lon,next_lat,next_lon = getCoordinates(next(x))
            x1 = p.apply_async(getRow,[nfis_tif,year,lat,lon,next_lat,next_lon])
            lat,lon,next_lat,next_lon = getCoordinates(next(x))
            print(lat,lon)
            x2 = p.apply_async(getRow,[nfis_tif,year,lat,lon,next_lat,next_lon])
            lat,lon,next_lat,next_lon = getCoordinates(next(x))

            x3 = p.apply_async(getRow,[nfis_tif,year,lat,lon,next_lat,next_lon])
            lat,lon,next_lat,next_lon = getCoordinates(next(x))

            x4 = p.apply_async(getRow,[nfis_tif,year,lat,lon,next_lat,next_lon])            
            lat,lon,next_lat,next_lon = getCoordinates(next(x))

            x5 = p.apply_async(getRow,[nfis_tif,year,lat,lon,next_lat,next_lon])
        print(x1.get())
        print(x2.get())
        print(x3.get())
        print(x4.get())
        print(x5.get())


44.76439666748047 -65.0


TypeError: getRow() missing 1 required positional argument: 'next_lon'